# **Portfolio Optimiser**
## **Contents**
1. [Introduction](#Introduction)
2. [Portfolio Metrics](#Modern-Portfolio-Theory)
   - [Portfolio Expected Return](#Portfolio-Expected-Return)
   - [Portfolio Covariance](#Portfolio-Covariance)
   - [Portfolio Variance and Volatility](#Portfolio-Variance-and-Volatility)
   - [Portfolio Sharpe Ratio](#Portfolio-Sharpe-Ratio)
3. [Portfolio Weightings](#Portfolio-Weightings)
   - [Equal-Weighted Portfolio](#Equal-Weighted-Portfolio)
   - [Minimum Volatility Portfolio](#Minimum-Volatility-Portfolio)
   - [Maximum Expected Return Portfolio](#Maximum-Expected-Return-Portfolio)
   - [Maximum Sharpe Ratio Portfolio](#Maximum-Sharpe-Ratio-Portfolio)
   - [Inverse-Volatility-Weighted Portfolio](#Minimum-Volatility-Portfolio)
4. [Efficient Frontier Construction](#Efficient-Frontier-Construction)
5. [Credits](#Credits)


## **Introduction**
This project aims to offer a set portfolio optimization tools for portfolios with risk-bearing financial assets. In particular, several portfolio metrics are defined, as are several different portfolio weightings, where the some of the latter are deduced as solutions to optimization problems. The efficient frontier of portfolios is also introduced and constructed. The `time`, `datetime`, `numpy`, `scipy`, `pandas`, `yfinance`, `matplotlib` and `seaborn` libraries are imported for all the numerical calculations and data analysis/visualisation techniques needed for the purposes of this project. Moreover, the plot styles are set for aesthetic reasons. Finally, all portfolios are computed with the constraint that no short-selling is allowed (for simplicity).

In [29]:
import time as tm
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style(style = "whitegrid", rc = {"font.family":"Times New Roman", "font.weight":"bold"})

## **Portfolio Metrics**
Let $(\Omega, \mathscr{F}, \mathbb{P})$ be a probability space, let $N$ be a non-zero natural number in $\mathbb{N}^{*}$, let $\mathbf{\vec{R}}$ be a $\mathbb{R}_{+}^{N}$-valued random vector on $(\Omega, \mathscr{F}, \mathbb{P})$, and let $\mathbf{\vec{w}}$ be a Euclidean vector in $[0, 1]^{N}$ such that $\sum_{k=1}^{N}w_{k} = 1$. Moreover, consider a portfolio of $N$ risk-bearing assets, having rates of return accodring to the Euclidean vector $\mathbf{\vec{R}}$ and respectively distributed according to the components of the Euclidean vector $\mathbf{\vec{w}}$.

### **Portfolio Expected Return**
- The *rate of return* of the portfolio is the random variable denoted by $R$ and defined by:
$$\forall \omega \in \Omega, \enspace R(\omega) = \mathbf{\vec{w}} \bullet \mathbf{\vec{R}}(\omega) = \sum_{k = 1}^{N}w_{k}\cdot R_{k}(\omega)$$
- The *expected return vector* of the portfolio is the Euclidean vector denoted by $\mathbf{\vec{\mu}}$ and defined by:
$$\mathbf{\vec{\mu}} = \mathbb{E}\left(\mathbf{\vec{R}}\right) = \left(\mathbb{E}(R_{k})\right)_{k=1}^{N}$$
- The *expected rate of return* of the portfolio is the real number denoted by $\mu$ and defined by:
$$\mu = \mathbf{\vec{w}} \bullet \mathbf{\vec{\mu}} = \sum_{k = 1}^{N}w_{k}\cdot \mu_{k} = \sum_{k = 1}^{N}w_{k}\cdot \mathbb{E}(R_{k})$$

### **Portfolio Covariance**
- The *return covariance* of the portfolio is the matrix denoted by $\mathbf{\Sigma}$ and defined by:
$$\mathbf{\Sigma} = \left[\mathbf{Cov}\left(R_{j}, R_{k}\right)\right]_{(j, k) \in \llbracket 1, N \rrbracket^{2}}$$

### **Portfolio Variance and Volatility**
- The *variance* of the portfolio is the positive real number denoted by $\mathbb{V}\left(R\right)$ and defined by:
$$\mathbb{V}\left(R\right) = \mathbf{\vec{w}} \times \mathbf{\Sigma}\times\mathbf{\vec{w}}^{\mathsf{T}}$$
- The *volatility* of the portfolio is the positive real number denoted by $\sigma\left(R\right)$ and defined by:
$$\sigma\left(R\right) = \sqrt{\mathbb{V}\left(R\right)}$$

### **Portfolio Sharpe Ratio**
Let $(r, \rho)$ be an ordered pair of strictly positive real numbers in $\mathbb{R}_{+}^{\ast} \times ]0, 1[$, suppose that the portfolio contains, in addition to the $N$ original risk-bearing assets, one riskless asset with rate of return $r$ and having proportion $\rho$ in the portfolio distribution of assets and let $\mathbf{\vec{v}}$ be the Euclidean vector representing the newly constructed portfolio's distribution of assets, defined by:
$$\mathbf{\vec{v}} = \left(\left(1 - \rho\right) \cdot w_{1}, \left(1 - \rho\right) \cdot w_{2}, \dots, \left(1 - \rho\right) \cdot w_{n}, \rho\right)$$
- The *Sharpe ratio* of the newly-constructed portfolio is the positive real number denoted by $\mathrm{Sharpe}\left(R, r\right)$ and defined by:
$$\mathrm{Sharpe}\left(R, r\right) = \frac{\mathbb{E}\left(R - r\right)}{\sigma\left(R - r\right)}$$

The Python implementation of the above metrics is given in the code cell below, using stock price data for Apple, Google and Microsoft over the year ending in the present day, where the assets are distributed equally in the portfolio and, in the case of the Sharpe ratio, imagining some riskless asset with expected return rate given by $\mathbb{E}(r) = 0.02 = 2\%$ (say).

In [32]:
tickers = ["AAPL", "GOOG", "MSFT"]
N = len(tickers)
w = np.ones(N) / N
end = dt.datetime.now()
start = end - 5 * pd.to_timedelta("365 days")
data = yf.download(tickers = tickers, start = start, end = end)
S = data["Close"]
r = 0.02
R = S.pct_change()
Mu = R.mean()
Sigma = R.cov()
V = np.dot(np.dot(w, Sigma), w.T)
mu = np.dot(w, Mu)
sigma = np.sqrt(V)
Sharpe = (Mu - r).mean() / (Mu - r).std()
print(f"Expected return: {mu} \nVolatility: {sigma} \nSharpe ratio: {Sharpe}")

C:\Users\Zakar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]C:\Users\Zakar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\Zakar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex cons

Expected return: 0.00117859253435097 
Volatility: 0.01782842536656467 
Sharpe ratio: -92.91923180118813


## **Portfolio Weightings**
Given the Euclidean vector $\mathbf{\vec{R}}$ of rates of returns of the risky assets in the portfolio, as well as the Euclidean vector $\mathbf{\vec{w}}$ of weights in the distribution of assets in the portfolio, completely determine the expected return $\mu$ and the volatility $\sigma$ of the portfolio. By extension, they also uniquely determines the Sharpe ratio $\mathrm{Sharpe}(R, r)$ of the portfolio. Given the Euclidean vector $\mathbf{R}$, it is often desirable to select the value of the Euclidean vector $\mathbf{\vec{w}}$ in such a way that $\mu$ is as high as possible, or that $\sigma$ is as low as possible, pending certain conditions being satisfied. That is, it is desirable to select the value of the Euclidean vector $\mathbf{\vec{w}}$ which solves specific optimization problems in $\mathbf{\vec{w}}$, which are expressed in terms of $\mathbf{\vec{w}}$, $\mathbf{\Sigma}$, $\mu$, $\sigma$, and/or $r$. Such selections are *portfolio weightings* (or *portfolio allocations*, or *asset allocations*), of which several stand out.

### **Equal-Weighted Portfolio**
The equal-weighted portfolio is the portfolio wherein all components of the Euclidean vector $\mathbf{\vec{w}}$ are equal:
$$\mathbf{\vec{w}} = \left(\frac{1}{N}\right)_{k = 1}^{N}$$

### **Inverse-Volatility-Weighted Portfolio**
The inverse-volatility-weighted portfolio is the portfolio wherein the components of the Euclidean vector $\mathbf{\vec{w}}$ are inversely proportional to their corresponding asset volatilities:
$$\left(\forall k \in \llbracket 1, N \rrbracket, \enspace \sigma_{k} = \sigma\left(R_{k}\right)\right) \implies \mathbf{\vec{w}} = \left(\dfrac{\frac{1}{\sigma_{k}}}{\sum_{j=1}^{N}\frac{1}{\sigma_{j}}}\right)_{k = 1}^{N}$$

### **Maximum Expected Return Portfolio**
The maximum expected return portfolio is the portfolio wherein the Euclidean vector $\mathbf{\vec{w}}$ maximises the value of the expected return $\mu$ subject to an upper bound $\sigma_{\ast}$ on $\sigma$, that is, the Euclidean vector $\mathbf{\vec{w}}$ satisfies:
$$\mathbf{\vec{w}} \in \displaystyle \argmax_{\substack{\mathbf{\vec{x}} \in ]0, 1[^{N} \\ \sigma(\mathbf{\vec{x}}) \leqslant \sigma_{\ast} \\ \sum_{k=1}^{N}x_{k} = 1}}\left(\mu\left(\mathbf{\vec{x}}\right)\right)$$
or, equivalently, solves the following quadratic program:
\begin{align}
\textrm{maximise}   \quad   &   \mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\vec{\mu}} \\
\textrm{subject to} \quad   &   \mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\Sigma} \times \mathbf{\vec{w}} \leqslant \sigma_{\ast}^2, \\
                            &   \mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\vec{1}} = 1
\end{align}

### **Minimum Volatility Portfolio**
The minimum volatility portfolio is the portfolio wherein the Euclidean vector $\mathbf{\vec{w}}$ minimises the value of the volatility $\sigma$ subject to a lower bound $\mu_{\ast}$ on $\mu$, that is, the Euclidean vector $\mathbf{\vec{w}}$ satisfies:
$$\mathbf{\vec{w}} \in \displaystyle \argmin_{\substack{\mathbf{\vec{x}} \in ]0, 1[^{N} \\ \mu(\mathbf{\vec{x}}) \geqslant \mu_{\ast} \\ \sum_{k=1}^{N}x_{k} = 1}}\left(\sigma\left(\mathbf{\vec{x}}\right)\right)$$
or, equivalently, solves the following quadratic program:
\begin{align}
\textrm{minimise}   \quad   &   \mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\Sigma} \times \mathbf{\vec{w}} \\
\textrm{subject to} \quad   &   \mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\vec{\mu}} \geqslant \mu_{\ast}, \\
                            &   \mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\vec{1}} = 1
\end{align}

### **Maximum Sharpe Ratio Portfolio**
The maximum Sharpe Ratio portfolio is the portfolio wherein the Euclidean vector $\mathbf{\vec{w}}$ maximises the value of the Sharpe ratio $\mathrm{Sharpe}(R, r)$, that is, the Euclidean vector $\mathbf{\vec{w}}$ satisfies:
$$\mathbf{\vec{w}} \in \displaystyle \argmax_{\substack{\mathbf{\vec{x}} \in ]0, 1[^{N} \\ \sum_{k=1}^{N}x_{k} = 1}}\left(\mathrm{Sharpe}\left(R\left(\mathbf{\vec{x}}\right), r\right)\right)$$
or, equivalently, solves the following quadratic program:
\begin{align}
\textrm{maximise}   \quad   &   \frac{\mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\vec{\mu}} - r}{\sqrt{\mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\Sigma} \times \mathbf{\vec{w}}}} \\
\textrm{subject to} \quad   &   \mathbf{\vec{w}}^{\mathsf{T}} \times \mathbf{\vec{1}} = 1
\end{align}
